In [10]:
from sympy import *
import numpy as np
import pandas as pd
from copy import copy,deepcopy
import random

In [134]:
def cpAp(e,lst):
    a = np.append(e,lst)
    return a

def aptitud(pos,neg,notapt):
    pos_values = {}
    neg_values = {}
    onlykeys = {x[0] for x in notapt}
    for i in range(len(pos)):
        for j in range(len(pos[i])):
            if str(j) not in pos_values:
                pos_values[str(j)+",1"] = 0
                pos_values[str(j)+",0"] = 0
            if pos[i][j] == 1:
                key = str(j)+",1"
            else:
                key = "{},0".format(j)
            if key not in onlykeys:
                pos_values[key] += 1
    for i in range(len(neg)):
        for j in range(len(neg[i])):
            if str(j) not in neg_values:
                neg_values[str(j)+",1"] = 0
                neg_values[str(j)+",0"] = 0
            if neg[i][j] == 1:
                key = str(j)+",1"
            else:
                key = "{},0".format(j)
            if key not in onlykeys:
                neg_values[key] += 1
    res = []
    for key in pos_values:
        if neg_values[key] == 0 and pos_values[key] != 0:
            res.append([key,1000])
        elif neg_values[key] != 0:
            res.append([key,pos_values[key]/neg_values[key]])
    res.sort(key=lambda x: x[1],reverse=True)
    return res

def removeBest(pos,key):
    filtered = []
    key = key.split(",")
    index = int(key[0])
    value = int(key[1])
    for i in range(len(pos)):
        if pos[i][index] != value:
            filtered.append(pos[i])
    return filtered

def removeNeg(neg,t):
    newneg = []
    for i in range(len(neg)):
        add = False
        for j in range(len(t)):
            key = t[j]
            key = key.split(",")
            index = int(key[0])
            value = int(key[1])
            if neg[i][index] == value:
                add = True
                break
        if add:
            newneg.append(neg[i])
    return newneg

def ocat(data,m):
    neg = deepcopy(data["neg"])
    C = []
    while len(neg) > 0:
        t = []
        pos = deepcopy(data["pos"])
        notapt = []
        while len(pos) > 0:
            apt = aptitud(pos,neg,notapt)
            if m >= len(apt):
                notapt = []
            else:
                notapt = apt[m:]
                apt = apt[:m]
            best = apt[0]
            t.append(best[0])
            pos = removeBest(pos,best[0])
        C.append(t)
        neg = removeNeg(neg,t)
    return C

<img src="images/squaringCA.png"/>

Para cada uno de los atributos encontrar su funcion de trancision local
## cargamos el dataset del automata celular que calcula 2^2

In [18]:
dataframe = pd.read_csv("data/2_square.csv")
dataframe

,a1,a2,a3,a4,a5,a6,a7,a8
0,1,1,3,0,0,0,0,0
1,1,4,5,0,0,0,0,0
2,4,6,2,1,0,0,0,0
3,7,1,5,1,0,0,0,0
4,1,7,2,6,0,0,0,0
5,1,2,3,1,1,0,0,0
6,1,3,5,1,1,0,0,0
7,0,5,2,6,1,0,0,0
8,0,2,5,1,6,0,0,0
9,0,2,2,6,1,1,0,0


### Eliminamos las filas redundantes

In [21]:
dataframe = dataframe[:13]
dataframe

,a1,a2,a3,a4,a5,a6,a7,a8
0,1,1,3,0,0,0,0,0
1,1,4,5,0,0,0,0,0
2,4,6,2,1,0,0,0,0
3,7,1,5,1,0,0,0,0
4,1,7,2,6,0,0,0,0
5,1,2,3,1,1,0,0,0
6,1,3,5,1,1,0,0,0
7,0,5,2,6,1,0,0,0
8,0,2,5,1,6,0,0,0
9,0,2,2,6,1,1,0,0


In [46]:
#creamos un problema de aprendizaje para cada atributo
attr_state = {}
for i,row in dataframe.iterrows():
    for j in range(len(dataframe.columns)):
        if j not in attr_state:
            attr_state[j] = []
        x1 = 0
        x2 = row[j]
        x3 = 0
        if j != 0:
            x1 = row[j-1]
        if j != len(dataframe.columns)-1:
            x3 = row[j+1]
        attr_state[j].append([x1,x2,x3])
attr_state[0] '(A1 >= 1) and (A2 < 3 or A2 >= 4) and (A1 < 4 or A1 >= 7) and (A2 < 3 or A2 >= 5) --> 1'

[[0, 1, 1],
 [0, 1, 4],
 [0, 4, 6],
 [0, 7, 1],
 [0, 1, 7],
 [0, 1, 2],
 [0, 1, 3],
 [0, 0, 5],
 [0, 0, 2],
 [0, 0, 2],
 [0, 0, 2],
 [0, 0, 2],
 [0, 0, 2]]

In [72]:
#encontramos el dominio de cada atributo en los subproblemas de aprendizaje
domain = {}
for key in attr_state:
    for row in attr_state[key]:
        for j,col in enumerate(row):
            if key not in domain:
                domain[key] = {}
            if j not in domain[key]:
                domain[key][j] = []
            if col not in domain[key][j]:
                domain[key][j].append(col)
for k in domain:
    for j in domain[k]:
        domain[k][j].sort()
domain

{0: {0: [0], 1: [0, 1, 4, 7], 2: [1, 2, 3, 4, 5, 6, 7]},
 1: {0: [0, 1, 4, 7], 1: [1, 2, 3, 4, 5, 6, 7], 2: [2, 3, 5]},
 2: {0: [1, 2, 3, 4, 5, 6, 7], 1: [2, 3, 5], 2: [0, 1, 6]},
 3: {0: [2, 3, 5], 1: [0, 1, 6], 2: [0, 1, 6]},
 4: {0: [0, 1, 6], 1: [0, 1, 6], 2: [0, 1, 6]},
 5: {0: [0, 1, 6], 1: [0, 1, 6], 2: [0, 1]},
 6: {0: [0, 1, 6], 1: [0, 1], 2: [0]},
 7: {0: [0, 1], 1: [0], 2: [0]}}

In [111]:
#binarizamos los subproblemas de aprendizaje
binarized = {}
segments = {}
for key in attr_state:
    for row in attr_state[key]:
        temp = []
        if key not in binarized:
            binarized[key] = []
        if key not in segments:
            segments[key] = {}
        for j,col in enumerate(row):
            if len(domain[key][j]) > 1:
                if j not in segments[key]:
                    segments[key][j] = []
                if len(temp) not in segments[key][j]:
                    segments[key][j].append(len(temp))
                for k,val in enumerate(domain[key][j]):
                    if col >= val:
                        temp.append(1)
                    else:
                        temp.append(0)
                if len(temp)-1 not in segments[key][j]:
                    segments[key][j].append(len(temp)-1)
            else:
                temp.append(0)
        binarized[key].append(temp)
binarized[0]
segments

{0: {1: [1, 4], 2: [5, 11]},
 1: {0: [0, 3], 1: [4, 10], 2: [11, 13]},
 2: {0: [0, 6], 1: [7, 9], 2: [10, 12]},
 3: {0: [0, 2], 1: [3, 5], 2: [6, 8]},
 4: {0: [0, 2], 1: [3, 5], 2: [6, 8]},
 5: {0: [0, 2], 1: [3, 5], 2: [6, 7]},
 6: {0: [0, 2], 1: [3, 4]},
 7: {0: [0, 1]}}

In [137]:
# obtenemos los ejemplos positivos y negativos para el atributo x[1] de los subproblemas
pos_neg = {}
for key in attr_state:
    if key not in pos_neg:
        pos_neg[key] = {}
    for j,col in enumerate(domain[key][1]):
        if col not in pos_neg[key]:
            pos_neg[key][col] = {"pos":[],"neg":[]}
        for i,row in enumerate(attr_state[key]):
            if row[1] == col:
                if i > 0:
                    pos_neg[key][col]["pos"].append(binarized[key][i-1])
            else:
                if i > 0:
                    pos_neg[key][col]["neg"].append(binarized[key][i-1])
pos_neg[0]

{0: {'pos': [[0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0],
   [0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
   [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
   [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
   [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
   [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]],
  'neg': [[0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
   [0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0],
   [0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0],
   [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
   [0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1],
   [0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0]]},
 1: {'pos': [[0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
   [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
   [0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1],
   [0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0]],
  'neg': [[0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0],
   [0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0],
   [0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0],
   [0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
   [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
   [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
   [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
   [0, 1, 0, 0, 0, 

In [140]:
rules = {}
for key in pos_neg:
    if key not in rules:
        rules[key] = {}
    for cls in pos_neg[key]:
        if len(pos_neg[key][cls]["pos"]) > 0 and len(pos_neg[key][cls]["neg"]) > 0:
            if cls not in rules[key]:
                rules[key][cls] = []
            rules[key][cls] = ocat(pos_neg[key][cls],5)
rules

{0: {0: [['2,0', '7,1'], ['2,0', '8,0']],
  1: [['2,1'], ['7,0', '8,1'], ['3,0', '4,1'], ['7,0', '9,1']],
  4: [['2,1'], ['8,1'], ['9,0']],
  7: [['2,1'], ['3,1'], ['4,0']]},
 1: {1: [['1,1'], ['2,1'], ['3,0']],
  2: [['1,0', '7,1'], ['1,0', '2,0'], ['1,0', '8,1']],
  3: [['1,1'], ['6,0'], ['2,0'], ['5,1']],
  4: [['1,1'], ['5,0'], ['2,0']],
  5: [['1,1'], ['6,1'], ['7,0']],
  6: [['1,1'], ['7,1'], ['8,0']],
  7: [['1,1'], ['2,1'], ['3,1']]},
 2: {2: [['2,0', '3,0', '4,0'], ['8,0', '9,1']],
  3: [['2,1'], ['5,1'], ['6,1']],
  5: [['2,1', '8,1'],
   ['2,1', '9,0'],
   ['3,1', '2,0'],
   ['5,0', '6,0'],
   ['4,1', '2,0']]},
 3: {0: [['1,1'], ['2,0'], ['4,0']],
  1: [['1,0', '2,0', '4,0'], ['1,0', '4,1', '2,1']],
  6: [['1,1'], ['2,1'], ['4,1']]},
 4: {0: [['4,0'], ['2,0']],
  1: [['2,0', '4,0'], ['4,1', '2,1']],
  6: [['2,1'], ['4,1']]},
 5: {0: [['4,0'], ['2,0']],
  1: [['2,0', '4,0'], ['4,1', '2,1']],
  6: [['2,1'], ['4,1']]},
 6: {0: [['2,0']], 1: [['2,1']]},
 7: {}}

In [144]:
#Obtenemos las clausulas
selectors = {}
for key in rules:
    if key not in selectors:
        selectors[key] = {}
    for attr in rules[key]:
        if attr not in selectors[key]:
            selectors[key][attr] = ""
        ands = []
        for row in rules[key][attr]:
            ors = []
            for col in row:
                temp = col.split(",")
                index = int(temp[0])
                val = int(temp[1])
                selector = ""
                for ksect in segments[key]:
                    if index >= segments[key][ksect][0] and index <= segments[key][ksect][1]:
                        if val == 0:
                            selector += "A{} < {}".format(ksect,domain[key][ksect][index-segments[key][ksect][0]])
                        else:
                            selector += "A{} >= {}".format(ksect,domain[key][ksect][index-segments[key][ksect][0]])
                        break
                if selector != "":
                    ors.append(selector)
                else:
                    if val == 0:
                        ors.append("!A{}".format(index))
                    else:
                        ors.append("A{}".format(index))
            ands.append(" or ".join(ors))
        selectors[key][attr] =  "("+(") and (".join(ands))+") --> " + str(attr) 
selectors

{0: {0: '(A1 < 1 or A2 >= 3) and (A1 < 1 or A2 < 4) --> 0',
  1: '(A1 >= 1) and (A2 < 3 or A2 >= 4) and (A1 < 4 or A1 >= 7) and (A2 < 3 or A2 >= 5) --> 1',
  4: '(A1 >= 1) and (A2 >= 4) and (A2 < 5) --> 4',
  7: '(A1 >= 1) and (A1 >= 4) and (A1 < 7) --> 7'},
 1: {1: '(A0 >= 1) and (A0 >= 4) and (A0 < 7) --> 1',
  2: '(A0 < 1 or A1 >= 4) and (A0 < 1 or A0 < 4) and (A0 < 1 or A1 >= 5) --> 2',
  3: '(A0 >= 1) and (A1 < 3) and (A0 < 4) and (A1 >= 2) --> 3',
  4: '(A0 >= 1) and (A1 < 2) and (A0 < 4) --> 4',
  5: '(A0 >= 1) and (A1 >= 3) and (A1 < 4) --> 5',
  6: '(A0 >= 1) and (A1 >= 4) and (A1 < 5) --> 6',
  7: '(A0 >= 1) and (A0 >= 4) and (A0 >= 7) --> 7'},
 2: {2: '(A0 < 3 or A0 < 4 or A0 < 5) and (A1 < 3 or A1 >= 5) --> 2',
  3: '(A0 >= 3) and (A0 >= 6) and (A0 >= 7) --> 3',
  5: '(A0 >= 3 or A1 >= 3) and (A0 >= 3 or A1 < 5) and (A0 >= 4 or A0 < 3) and (A0 < 6 or A0 < 7) and (A0 >= 5 or A0 < 3) --> 5'},
 3: {0: '(A0 >= 3) and (A0 < 5) and (A1 < 1) --> 0',
  1: '(A0 < 3 or A0 < 5 or A1 <